# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [61]:
#Your code here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

In [107]:
homepage = pd.read_csv('homepage_actions.csv')
homepage.head(10)

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
5,2016-09-24 20:32:25.712659,681598,experiment,view
6,2016-09-24 20:39:03.248853,522116,experiment,view
7,2016-09-24 20:57:20.336757,349125,experiment,view
8,2016-09-24 20:58:01.948663,349125,experiment,click
9,2016-09-24 21:00:12.278374,560027,control,view


In [109]:
homepage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [110]:
# Check the summary statistics of the dataset
homepage.describe()

,id
count,8188.000000
mean,564699.749878
std,219085.845672
min,182988.000000
25%,373637.500000
50%,566840.500000
75%,758078.000000
max,937217.000000


In [111]:
# converting timestamp column from 
homepage['timestamp'] = pd.to_datetime(homepage['timestamp'])
print("The data Type for timestamp column is now: ",homepage['timestamp'].dtype)
print(homepage['timestamp'])

The data Type for timestamp column is now:  datetime64[ns]
0      2016-09-24 17:42:27.839496
1      2016-09-24 19:19:03.542569
2      2016-09-24 19:36:00.944135
3      2016-09-24 19:59:02.646620
4      2016-09-24 20:26:14.466886
                  ...            
8183   2017-01-18 09:11:41.984113
8184   2017-01-18 09:42:12.844575
8185   2017-01-18 10:01:09.026482
8186   2017-01-18 10:08:51.588469
8187   2017-01-18 10:24:08.629327
Name: timestamp, Length: 8188, dtype: datetime64[ns]


In [122]:
clicks = set(homepage[homepage.action=='click']['id'].unique())
views = set(homepage[homepage.action=='view']['id'].unique())
print("The  total number of viewers:",len(views))
print("The number of viewers who clicked the link:",len(clicks))
print("The number of viewers who didn't click the link:",len(views - clicks))
print("The number of clickers who didn't view the link:",len(clicks - views))

The  total number of viewers: 6328
The number of viewers who clicked the link: 1860
The number of viewers who didn't click the link: 4468
The number of clickers who didn't view the link: 0


In [118]:
num_overlapping_ids = len(set(homepage[homepage.group == 'experiment']['id']) & set(homepage[homepage.group == 'control']['id']))
print(f"Overlap of experiment and control groups: {num_overlapping_ids}")

Overlap of experiment and control groups: 0


There was a total of 6328 viewers, of which 1860 clicked the link, while 4468  did not. lucky there were no clickers who did not view the site. We also did not register any overlap of the data

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

Null Hypothesis (H0): The experimental homepage is not more effective than the control homepage.

Alternative Hypothesis (Ha): The experimental homepage is more effective than the control homepage.

We can use a one-tailed t-test to test this hypothesis. First, we need to calculate the sample means and sample standard deviations of the conversion rates for the experimental and control groups. Then, we can calculate the t-statistic and the corresponding p-value

In [125]:
# Calculating click through rate using Pivot table
# Column for counting clicks
homepage['count'] = 1

# Control Dataframe
control = homepage[homepage['group'] == 'control']
# Experment Dataframe
experiment = homepage[homepage['group'] == 'experiment']

# Pivot tables 
# control pivot
control = control.pivot_table(index='id', columns='action', values='count', fill_value=0)
# experiment pivot
experiment = experiment.pivot_table(index='id', columns='action', values='count', fill_value=0)

print(control.head())
print(experiment.head())

action  click  view
id                 
182994      1     1
183089      0     1
183248      1     1
183515      0     1
183524      0     1
action  click  view
id                 
182988      0     1
183136      0     1
183141      1     1
183283      0     1
183389      0     1


In [126]:
# Control group statistics
control_sample_size = len(control)
control_total_clicks = control['click'].sum()
control_click_rate = control['click'].mean()

# Experiment group statistics
experiment_sample_size = len(experiment)
experiment_total_clicks = experiment['click'].sum()
experiment_click_rate = experiment['click'].mean()

# Print statistics
print("Control group sample size: {}".format(control_sample_size))
print("Control group total clicks: {}".format(control_total_clicks))
print("Control group click rate: {:.4f}".format(control_click_rate))
print("\nExperiment group sample size: {}".format(experiment_sample_size))
print("Experiment group total clicks: {}".format(experiment_total_clicks))
print("Experiment group click rate: {:.4f}".format(experiment_click_rate))


Control group sample size: 3332
Control group total clicks: 932
Control group click rate: 0.2797

Experiment group sample size: 2996
Experiment group total clicks: 928
Experiment group click rate: 0.3097


In [141]:
import flatiron_stats as fs

fs.p_value_welch_ttest(control.click, experiment.click)

0.004466402814337078

In [142]:
# Interpret results
alpha = 0.05
if p_value < alpha:
    print("\nWe reject the null hypothesis.")
    print("The experimental homepage is more effective than the control homepage.")
else:
    print("\nWe fail to reject the null hypothesis.")
    print("There is insufficient evidence to conclude that the experimental homepage is more effective than the control homepage.")


We reject the null hypothesis.
The experimental homepage is more effective than the control homepage.


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [143]:
#Your code here
expected_experiment_clicks = control_click_rate * experiment_sample_size
print("Expected number of clicks in the experiment group: {:.2f}".format(expected_experiment_clicks))

Expected number of clicks in the experiment group: 838.02


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [148]:
#Your code here
n = len(experiment)
p = control_click_rate
var = n * p * (1-p)
std = np.sqrt(var)
print(std)

24.568547907005815


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [151]:
#Your code here
from scipy.stats import norm
z_score = ((experiment['click'].sum()) - expected_experiment_clicks) / std
p_value = 2 * (1 - norm.cdf(abs(z_score)))

print("\nZ-score: {:.2f}".format(z_score))
print("Two-tailed p-value: {:.4f}".format(p_value))


Z-score: 3.66
Two-tailed p-value: 0.0002


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **we have evidence to reject the null hypothesis and conclude that the click-through rates for the experiment and control groups are different.**

**The p-value calculated here matches the result of the previous statistical test, where we used bootstrapping to estimate the p-value. Both methods agree in rejecting the null hypothesis, which strengthens our conclusion about the difference in click-through rates between the two groups.**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.